## Projet 2, TP2 - Perceptron et classification de MNIST

Ce TP est une version édulcorée du [tutoriel PyTorch](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html), en nous concentrant sur quelques aspects mathématiques plutôt que la syntaxe PyTorch pure.

In [52]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

from utils import DEFAULT_SEED, torch_rng

torch.manual_seed(DEFAULT_SEED);

## Partie 1 - Le jeu de données

Le jeu de données pèse quelques Mo. On le télécharge seulement à la première exécution (l'argument `transform=ToTensor()` permet simplement de manipuler des tenseurs plutôt que des images dans le reste du TP).

In [87]:
from torchvision import datasets
from torchvision.transforms import ToTensor

dataset = datasets.MNIST("data", download=True, transform=ToTensor())

### 1.1. Étude du dataset

Comme dans le cas des la régression, le dataset consiste en des couples (entrée, sortie). Quelles sont les entrées ? Quelles sont les sorties ?

### 1.2. Affichage de quelques images avec leur étiquette

In [ ]:
rng = torch_rng(seed=3373759423)  # seed choisie pour être un peu intéressante

figure = plt.figure(figsize=(3, 3))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    # tirage aléatoire d'un indice
    sample_idx = torch.randint(len(dataset), size=(1,), generator=rng).item()
    
    # TODO extraction de l'image et de l'étiquette

    # affichage 
    figure.add_subplot(rows, cols, i)
    plt.axis("off")

    # TODO mettre l'étiquette en titre et afficher l'image avec `plt.imshow`

# affichage de la figure
plt.tight_layout()
plt.show()

## Partie 2 - Le réseau de neurones

Comment représenter la sortie du réseau et choisir la loss ? Un nombre entre 0 et 9, qu'on compare avec l'étiquette ? Cela semblerait naturel, mais se confronte vite à des limites. D'abord, parce que cette approche «naïve» pénalise les erreurs de prédiction bizarrement. Est-il normal, au moment d'une prédiction d'un «9», de plus pénaliser une classification en «0» qu'en «8» ? Dans ce contexte, pas vraiment. En outre, ce processus est incompatible avec des étiquettes qui ne seraient pas numériques. Et même si on associait à chaque nombre un nombre, comment choisir ce nombre, surtout au vu de la remarque précédente ?

Face à ces limites, on se voit forcé de réfléchir différement à la sortie du réseau. Ici, on va le voir comme des intensités dans un **espace latent**, à savoir un vecteur de taille 10, chaque coordonnée correspondant à une classe. On convertit ensuite ces intensités en probabilités : une estimation du type «je pense que c'est un 5 à 90%, mais ça ressemble aussi un peu à un 6 à 10%». 
Pour cela, on utilise la fonction [Softmax](https://youtu.be/wjZofJX0v4M?si=y5VwxM2BfUD-u0lb&t=1324).
Pour éviter de complètement négliger les petites probabilités et comparer ce qui est comparable, on prend le logarithme de ces probabilités.

La structure de réseau est donc la suivante :
1. Applatissement de l'image avec une couche `Flatten` pour convertir la matrice en vecteur ;
2. Un perceptron multi-couches avec 2 couches cachées de dimension 512, des fonctions d'activation `ReLU` et une sortie en dimension 10 ;
3. Conversion des intensités en log-probabilités avec une couche `LogSoftmax`.

**TODO**  
Définir une classe correspondant à ce réseau de neurones.

### 2.2. Validation préliminaire du réseau

**TODO**  
Sur un point de données, vérifier que toutes les sorties sont négatives (des log de données inférieures à 1), et que la somme des exponentielles est égale à 1.

## Partie 3 - Apprentissage !

### 3.1. Choix de la loss

Pour la loss, on se base sur le principe du **maximum de vraisemblance** (en l'occurrence la log-vraisemblance). Ainsi, la loss qu'on cherche à minimiser est 
$$ \ell(c, y) = -c_y = -\log(p_y), \qquad c = (\log(p_0), \log(p_1), ..., \log(p_9))^{\sf T}, $$
qui prend à gauche un vecteur de log-probabilités, et à droite l'indice de la classe. Celle-ci est implémentée avec [`nn.NLLLoss`](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html).

*Remarque :* On pourrait aussi ne pas convertir les intensités (*logits*) en log-probabilités et utiliser la *cross-entropy loss* qui ferait cette conversion automatiquement. Les deux approches sont équivalentes.

**TODO**  
Définir la loss et la calculer sur le point de données précédent.

### 3.2. La boucle d'apprentissage

Puisque la loss sera difficile à interpréter, calculer en outre la **précision**, pour l'afficher au cours de l'entraînement. Pour calculer la précision, on détermine la classe la plus probable dans la sortie, et on vérifie si elle correspond à l'étiquette connue. À la fin de l'époque, on évalue quel pourcentage de telles prédictions étaient correctes. 

**Attention**, cela n'empêche pas de calculer la loss sur laquelle on effectue la rétropropagation !

**TODO**  
Écrire une boucle d'apprentissage qui, tous les 100 batches, affiche la loss et la précision sur les 100 batches précédents.

### 3.3. Effectuer l'apprentissage

**TODO**  
Effectuer un apprentissage sur des lots de taille 64, avec l'algorithme d'optimisation Adam et un *learning rate* de $10^{-3}$, sur 5 époques.

### 3.4. Étude qualitative de l'apprentissage

**TODO**  
Afficher quelques images avec leur étiquette estimée par le réseau.